# Проект. Классификация комментариев

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. Имеется набор данных с разметкой о токсичности правок.

Нужно построить модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## 1. Подготовка данных

Импортируем необходимые модули и библиотеки.

In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
import numpy as np
import re
from scipy.sparse import hstack
from catboost import Pool, CatBoostClassifier
from catboost.text_processing import Tokenizer
from sklearn import linear_model
from itertools import product

Сохраним данные в переменной `data`.

In [2]:
data = pd.read_csv('./toxic_comments.csv')

Изучим информацию о данных.

In [3]:
data.tail()

,text,toxic
159566,""":::::And for the second time of asking, when ...",0
159567,You should be ashamed of yourself \n\nThat is ...,0
159568,"Spitzer \n\nUmm, theres no actual article for ...",0
159569,And it looks like it was actually you who put ...,0
159570,"""\nAnd ... I really don't think you understand...",0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [5]:
data['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

* Пропусков в данных нет. В целевых признаках есть дисбаланс классов.

Сохраним признаки в переменной `features`, а целевой признак в переменной `target`.

In [6]:
features = data['text']
target = data['toxic']

Разделим выборки на тренировочную и тестовую в соотношении один к одному, причем используем аргумент `stratify`, чтобы в обеих выборках было одинаковое соотношения классов.

In [7]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.5, stratify=target)

Посмотрим на размеры выборок.

In [8]:
print('Тренировочная выборка (признаки):',features_train.shape)
print('Тестовая выборка (признаки):',features_test.shape)
print('Тренировочная выборка (целевые признаки):',target_train.shape)
print('Тестовая выборка (целевые признаки):',target_test.shape)
print('Соотношение классов в целевых признаках тренировочной выборки:\n', target_train.value_counts())
print('Соотношение классов в целевых признаках тестовой выборки:\n', target_test.value_counts())

Тренировочная выборка (признаки): (79785,)
Тестовая выборка (признаки): (79786,)
Тренировочная выборка (целевые признаки): (79785,)
Тестовая выборка (целевые признаки): (79786,)
Соотношение классов в целевых признаках тренировочной выборки:
 0    71673
1     8112
Name: toxic, dtype: int64
Соотношение классов в целевых признаках тестовой выборки:
 0    71673
1     8113
Name: toxic, dtype: int64


## 2. Обучение моделей

### 2.1 Логистическая регрессия.

В качестве признаков для обучения логистической регрессии используем величины TF-IDF, которые получим с помощью класса `TfidfVectorizer`. Причем получим эти величины для слов, для символов и для n-gram слов и символов, а затем объединим эти данные в один датасет.

In [9]:
count_tf_idf_word = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1))

In [10]:
tf_idf_train_word = count_tf_idf_word.fit_transform(features_train)

In [11]:
tf_idf_test_word = count_tf_idf_word.transform(features_test)

In [12]:
count_tf_idf_char = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(1, 2))

In [13]:
tf_idf_train_char = count_tf_idf_char.fit_transform(features_train)

In [14]:
tf_idf_test_char = count_tf_idf_char.transform(features_test)

In [15]:
count_tf_idf_char_wb = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char_wb',
    stop_words='english',
    ngram_range=(1, 2))

In [16]:
tf_idf_train_char_wb = count_tf_idf_char_wb.fit_transform(features_train)

In [17]:
tf_idf_test_char_wb = count_tf_idf_char_wb.transform(features_test)

Объединяем получившиеся данные тренировочную и тестовую выборки.

In [18]:
tf_idf_train = hstack([tf_idf_train_char, tf_idf_train_word, tf_idf_train_char_wb])

In [19]:
tf_idf_test = hstack([tf_idf_test_char, tf_idf_test_word, tf_idf_test_char_wb])

Посмотрим на размеры получившихся выборок.

In [20]:
print('Тренировочная выборка:', tf_idf_train.shape)
print('Тестовая выборка:', tf_idf_test.shape)

Тренировочная выборка: (79785, 146788)
Тестовая выборка: (79786, 146788)


Обучим линейную регрессию и посчтитаем метрику F1.

In [21]:
model = LogisticRegression(solver='liblinear', penalty='l1').fit(tf_idf_train, target_train)

In [22]:
pred = model.predict(tf_idf_test)

In [23]:
F1 = f1_score(pred, target_test)
print('F1 = {:.2f}'.format(F1))

F1 = 0.79


### 2.2 SGDClassifier

Попробуем также модель SGDClassifier. Подберем параметры с помощью итератора `product`.

In [24]:
gen = product([{'tol':0.001}, {'tol':0.0001},],
              [{'eta0':0.01},{'eta0':0.02},{'eta0':0.03},{'eta0':0.04}])

In [25]:
buf = dict() # создаем словарь для сохранения результата

for i in gen:
    print(i[0])
   
    model = linear_model.SGDClassifier(**i[0], **i[1], alpha=0.000001,
                                       random_state=12345, learning_rate='adaptive')
    
    model.fit(tf_idf_train, target_train)
    
    pred = model.predict(tf_idf_test)
    F1 = f1_score(pred, target_test)
    
    # Записываем результат вычисления метрики в словарь
    buf[str(i)] = 'F1 = {:.2f}'.format(F1)

{'tol': 0.001}
{'tol': 0.001}
{'tol': 0.001}
{'tol': 0.001}
{'tol': 0.0001}
{'tol': 0.0001}
{'tol': 0.0001}
{'tol': 0.0001}


Посмотрим что получилось.

In [26]:
buf

{"({'tol': 0.001}, {'eta0': 0.01})": 'F1 = 0.78',
 "({'tol': 0.001}, {'eta0': 0.02})": 'F1 = 0.79',
 "({'tol': 0.001}, {'eta0': 0.03})": 'F1 = 0.80',
 "({'tol': 0.001}, {'eta0': 0.04})": 'F1 = 0.80',
 "({'tol': 0.0001}, {'eta0': 0.01})": 'F1 = 0.79',
 "({'tol': 0.0001}, {'eta0': 0.02})": 'F1 = 0.79',
 "({'tol': 0.0001}, {'eta0': 0.03})": 'F1 = 0.79',
 "({'tol': 0.0001}, {'eta0': 0.04})": 'F1 = 0.78'}

Обучим модель на оптимальных гиперпараметрах.

In [27]:
model = linear_model.SGDClassifier(tol=0.001, eta0=0.04, alpha=0.000001, 
                                   random_state=12345, learning_rate='adaptive').fit(tf_idf_train, target_train)

In [28]:
pred = model.predict(tf_idf_test)

In [29]:
F1 = f1_score(pred, target_test)
print('F1 = {:.2f}'.format(F1))

F1 = 0.80


### 2.3 CatBoost Classifier

Попробуем модель CatBoost Classifier. Для того, чтобы передать аргумент `text_features`, создадим датафреймы обучающей и тестовой выборок с колонками *'text'*.

In [30]:
features_train = pd.DataFrame(data=features_train, columns=['text'])

In [31]:
features_test = pd.DataFrame(data=features_test, columns=['text'])

Напишем функцию `fit_catboost`.

In [32]:
def fit_catboost(X_train, X_test, y_train, y_test, catboost_params={}, verbose=100):
    
    # используем конструктор Pool для создания обучающей и тестовой выборки, обозначаем текстовые данные
    learn_pool = Pool(
        X_train, 
        y_train, 
        text_features=['text'],
    )
    test_pool = Pool(
        X_test, 
        y_test, 
        text_features=['text'],
    )
    
    # задаем гиперпараметры по умолчанию
    catboost_default_params = {
        'iterations': 1000,
        'learning_rate': 0.03,
        'eval_metric': 'F1',
        'task_type': 'GPU'
    }
    
    # обновляем дефолтные параметры параметрами из catboost_params
    catboost_default_params.update(catboost_params)
    
    # создаем и обучаем модель
    model = CatBoostClassifier(**catboost_default_params)
    model.fit(learn_pool, eval_set=test_pool, verbose=verbose)

    return model

Обучаем модель.

In [33]:
fit_catboost(
    features_train,
    features_test, 
    target_train, 
    target_test,
    
    # подбираем параметры для преобразования текстовых данных в числовые
    catboost_params={
        'text_processing': {
            # делаем токенизацию по словам и знакам пунктуации
            "tokenizers" : [{
                "tokenizer_id" : "Space",
                "delimiter" : " ",
                "lowercasing" : "true"
                },{
                "tokenizer_id" : "Punc",
                "delimiter" : " ",
                "lowercasing" : "true",
                "token_types" : "Punctuation"
            }],
            
            # создаем словари из 1-, 2- и 3-грамм 
            "dictionaries" : [{
                "dictionary_id" : "BiGram",
                "gram_order" : "2"
            },{
                "dictionary_id" : "Word",
                "gram_order" : "1"
            },{
                "dictionary_id" : "ThreeGram",
                "gram_order" : "3"
            }
            ],
            
            # применяем модели BoW NaiveBayes BM25 для создания числовых данных
            "feature_processing" : {
                "default" : [{
                    "dictionaries_names" : ["Word", "BiGram", "ThreeGram"],
                    "feature_calcers" : ["BoW", 'NaiveBayes', "BM25"],
                    "tokenizers_names" : ["Space", "Punc"]
                    }]
            }
        }
    }
)

0:	learn: 0.6677586	test: 0.6990667	best: 0.6990667 (0)	total: 152ms	remaining: 2m 31s
100:	learn: 0.6689785	test: 0.6988283	best: 0.7026839 (6)	total: 11.6s	remaining: 1m 43s
200:	learn: 0.6822838	test: 0.7072821	best: 0.7079215 (189)	total: 23s	remaining: 1m 31s
300:	learn: 0.6916398	test: 0.7146937	best: 0.7148669 (299)	total: 33.4s	remaining: 1m 17s
400:	learn: 0.6972069	test: 0.7164669	best: 0.7169165 (383)	total: 43.7s	remaining: 1m 5s
500:	learn: 0.7027844	test: 0.7175899	best: 0.7176303 (492)	total: 53.6s	remaining: 53.4s
600:	learn: 0.7064591	test: 0.7174830	best: 0.7177143 (547)	total: 1m 3s	remaining: 42.2s
700:	learn: 0.7098031	test: 0.7188103	best: 0.7189019 (699)	total: 1m 13s	remaining: 31.4s
800:	learn: 0.7125658	test: 0.7187589	best: 0.7189019 (699)	total: 1m 23s	remaining: 20.7s
900:	learn: 0.7149458	test: 0.7196796	best: 0.7199028 (882)	total: 1m 33s	remaining: 10.2s
999:	learn: 0.7170111	test: 0.7195540	best: 0.7199600 (985)	total: 1m 42s	remaining: 0us
bestTest = 0

# 3. Выводы

В ходе проекта удалось достичь значения метрики F1 = 0.8 на обучении модели SGDClassifier. В модели CatBoost удалось достичь значения метрики 0.72, но на вход подавался необработанный текст.

------